In [ ]:
# Creating dataset for PEER Illinois Funding Tool app

# Authors:  Chris Poulos (cdpoulos@gmail.com)
#           Erykah Nava (EMAIL)

# Purpose:  Read in, clean, and join the most recent Evidence-based Funding (EBF) 
#           distribution calculation data, Illinois Report Card (IRC) data, and
#           Employment Information System (EIS) salary data.

# Data sources:
#
# - EBF distribution calculation (EBF): https://www.isbe.net/ebfdist
# - Illinois Report Card (IRC): https://www.isbe.net/Pages/Illinois-State-Report-Card-Data.aspx
# - Employment Information System teacher salary dataset (EIS): https://www.isbe.net/Pages/Educator-Employment-Information.aspx

# NOTE on data sources.
#
# EBF data will be used for adequacy targets.
# IRC data will be used for demographic data and actual position counts.
# EIS data will be used for actual position counts when they aren't available in IRC.

In [ ]:
# Step 1 - Introduction, load packages, set working directory

# Import necessary packages

import pandas as pd
import os
import numpy as np

# Set working directory to location of data files

data_file_path = "G:\Shared drives\PEER IL Shared Drive\Data\data" # Change this to the directory with the data

file_path = rf"{data_file_path}"

<>:11: SyntaxWarning: invalid escape sequence '\S'
<>:11: SyntaxWarning: invalid escape sequence '\S'
C:\Users\cdpou\AppData\Local\Temp\ipykernel_14728\430541212.py:11: SyntaxWarning: invalid escape sequence '\S'
  data_file_path = "G:\Shared drives\PEER IL Shared Drive\Data\data" # Change this to the directory with the data


In [ ]:
# Step 2 - Read in data sets and inspect sheet names

# NOTE: Our data was saved in root/data/raw. Change the file path to reflect where your data is downloaded.

ebf_file = pd.ExcelFile(rf"{file_path}\raw\FY26-EBF-Full-Calc.xlsx") 
irc_file = pd.ExcelFile(rf"{file_path}\raw\2025-Report-Card-Public-Data-Set.xlsx")
ssp_file = pd.ExcelFile(rf"{file_path}\raw\School-Support-Personnel-Report-2024.xlsx")
eis_file = pd.ExcelFile(rf"{file_path}\raw\2024-ATSB-Report.xlsx")

# Print sheet names to inspect

# This helps identify which sheets to read in for each data set

print("EBF:")
print(ebf_file.sheet_names)
print("IRC:")
print(irc_file.sheet_names)
print("School Support Personnel:")
print(ssp_file.sheet_names)
print("Employment Information System:")
print(eis_file.sheet_names)


EBF:
['District Results Summary', 'Base Calc', 'Control Variables', 'Core Investments', 'Per Student Investments', 'Additional Investments', 'Local Capacity Target', 'Tier $', 'ENROLL ALL', 'ENROLL LI', 'ENROLL EL', 'ROE LOW INCOME', 'REGION FACTOR', 'ADJUSTED OTR', 'ADJUSTED EAV', 'REAL EAV CALC', 'PTELL EAV CALC', 'NEW ROE BFM', 'FY 26 BFM']
IRC:
['Revision History', 'General', 'General (2)', 'ACT', 'DLM', 'DLM (2)', 'IAR', 'ISA', 'ELAMathScience', 'ELAMathScience (2)', 'CTE', 'CTE (2)', 'Discipline', 'Finance', 'KIDS', 'SPED']
School Support Personnel:
['Districts', 'State']
Employment Information System:
['Sheet1']
Directory of Educational Entities:
['Category Guide', '1 Public Dist & Sch', '2 Spec Ed Dist & Sch', '3 ROE, ISC, Reg Prg', '4 CTE Area Career Ctr & EFE', '5 Non Pub Sch', '6 Non Pub Spec Ed', '7 Detention Ctr']


In [ ]:
# Step 3 - Create data frames using relevant sheets

# NOTE: EBF core investments and additional investments are the only sheets that contain personnel adequacy target counts
# NOTE: EBF base calc will be used for adequacy target and gap funding amounts

eci_x = pd.read_excel(rf"{file_path}\raw\FY26-EBF-Full-Calc.xlsx", sheet_name = "Core Investments")
ebc_x = pd.read_excel(rf"{file_path}\raw\FY26-EBF-Full-Calc.xlsx", sheet_name = "Base Calc")
eai_x = pd.read_excel(rf"{file_path}\raw\FY26-EBF-Full-Calc.xlsx", sheet_name = "Additional Investments")
irc_x = pd.read_excel(rf"{file_path}\raw\2025-Report-Card-Public-Data-Set.xlsx", sheet_name = "General")
rcf_x = pd.read_excel(rf"{file_path}\raw\2025-Report-Card-Public-Data-Set.xlsx", sheet_name = "Finance")
ssp_x = pd.read_excel(rf"{file_path}\raw\School-Support-Personnel-Report-2024.xlsx", sheet_name = "Districts")
eis_x = pd.read_excel(rf"{file_path}\raw\2024-ATSB-Report.xlsx", sheet_name = "Sheet1")

# Keep only the variables 'df' and 'file_path'. This is done to save memory.

for var in list(globals().keys()):
    if var not in ['eci_x', 'ebc_x', 'eai_x', 'irc_x', 'rcf_x','ssp_x', 'eis_x', 'file_path', 'pd', 'os', 'np', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython']:
        del globals()[var]

In [ ]:
# Step 4 - Copy df in case something goes wrong -- you don't need to take the time loading in the original data

eci = eci_x.copy()
ebc = ebc_x.copy()
eai = eai_x.copy()
irc = irc_x.copy()
rcf = rcf_x.copy()
ssp = ssp_x.copy()
eis = eis_x.copy()


In [ ]:
# Step 5 - Delete uncessary rows and inspect column name.

# NOTE: Open the explore data frames in data wrangler to make sure the code below still works. Change as necessary

# Get first and last row using column 2 for eci - "core investments"

col = eci.columns[1]  # Get the name of the second column

start_index = eci[eci[col] == "District Name"].index[0]
end_index = eci[eci[col] == "TOTALS"].index[0]

# Keep rows from start_index to end_index

eci = eci.loc[start_index+1:end_index]

# Repeat for ebc "base calc"

col = ebc.columns[1]

start_index = ebc[ebc[col] == "District Name"].index[0]
end_index = ebc[ebc[col] == "TOTALS"].index[0]

ebc = ebc.loc[start_index+1:end_index]

# Repeat for additional investments

col = eai.columns[1]

start_index = eai[eai[col] == "District Name"].index[0]
end_index = eai[eai[col] == "TOTALS"].index[0]

eai = eai.loc[start_index+1:end_index]

# The rows and columns in the other datasets do not need to be trimmed.

# Keep only the variables 'df' and 'file_path'

for var in list(globals().keys()):
    if var not in ['eci_x', 'ebc_x', 'eai_x', 'irc_x', 'rcf_x', 'ssp_x', 'eis_x','eci', 'ebc', 'eai', 'irc', 'rcf', 'ssp', 'dee','eis', 'file_path', 'pd', 'os', 'np', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython']:
        del globals()[var]


In [ ]:
# Step 6 - Keep only necessary columns and necessary changes to data frames.

## Evidence-based funding additional investments data frame

# Read in a copy of the data frame to keep the original intact.

eai_n = eai.copy()

# Keep necessary columns

eai_n = eai_n.iloc[:,[0,# RCDTS
                      9,# EBF Low Income Intervention Teacher Additional Investments
                      12,# EBF Low Income Pupil Support Staff Additional Investments 
                      15,# EBF Low Income Extended Day Teacher Additional Investments
                      18,# EBF Low Income Summer School Teacher Additional Investments
                      22,# EBF English Learner Intervention Teacher Additional Investments
                      25,# EBF English Learner Pupil Support Staff Additional Investments
                      28,# EBF English Learner Extended Day Teacher Additional Investments
                      31,# EBF English Learner Summer School Teacher Additional Investments
                      34,# EBF Core Teacher Additional Investments
                      38,# EBF Special Ed Teacher Additional Investments
                      41,# EBF Special Ed Instructional Assistant Additional Investments
                      44]]# EBF Special Ed School Psychologist Additional Investments

# Rename columns

eai_n.columns = ["RCDTS",
                 "EBF Low Income Intervention Teacher Additional Investments",
                 "EBF Low Income Pupil Support Staff Additional Investments",
                 "EBF Low Income Extended Day Teacher Additional Investments",
                 "EBF Low Income Summer School Teacher Additional Investments",
                 "EBF English Learner Intervention Teacher Additional Investments",
                 "EBF English Learner Pupil Support Staff Additional Investments",
                 "EBF English Learner Extended Day Teacher Additional Investments",
                 "EBF English Learner Summer School Teacher Additional Investments",
                 "EBF Core Teacher Additional Investments",
                 "EBF Special Ed Teacher Additional Investments",
                 "EBF Special Ed Instructional Assistant Additional Investments",
                 "EBF Special Ed School Psychologist Additional Investments"]

# Change to numeric: Loop through columns and convert to numeric ones not in exclude list

for col in eai_n.columns:
    if col != "RCDTS":
        eai_n[col] = pd.to_numeric(eai_n[col], errors='coerce')

# Add a statewide RCDTS value "6500000008000" which correpsonds to the IRC data 
# for the missing RCDTS value. NOTE: EBF data does not have a statewide RCDTS ID.

eai_n["RCDTS"] = eai_n["RCDTS"].fillna("6500000008000")

## Evidence-based funding base calc data frame

# Read in a copy of the data frame to keep the original intact.

ebc_n = ebc.copy()

# Keep necessary columns

ebc_n = ebc_n.iloc[:,[0, # RCDTS
                    3, # Organizational Type
                    5, # Total ASE
                    13, # Final Adequacy Target
                    14, # Final Adequacy Target Per Student
                    18, # Final Resources
                    21, # Adequacy Funding Gap
                    22, # Final Adequacy Level
                    ]]

# Rename columns

ebc_n.columns = ['RCDTS',
                 'EBF Base Calc Organization Type',
                 'Total ASE',
                 'EBF Base Calc Final Adequacy Target',
                 'EBF Base Calc Final Adequacy Target Per Student',
                 'EBF Base Calc Final Resources', 
                 'EBF Base Calc Adequacy Funding Gap', 
                 'EBF Base Calc Final Adequacy Level'
                 ]

# Reset ebc_n index

ebc_n = ebc_n.reset_index(drop=True)

# Add a state RCTDS code to the base calc dataframe.
# Add code to first missing row in RCDTS column.

replace = ebc_n["RCDTS"].count()

ebc_n.iloc[replace,0] = "6500000008000"
ebc_n.iloc[replace,1] = "State of Illinois"

# Drop remaining RCTDS missing rows

ebc_n = ebc_n[ebc_n["RCDTS"].notna()]

# Convert columns to numeric

# List of columns to exclude

exclude_cols = ["RCDTS","EBF Base Calc Organization Type"]

# Loop through columns and convert the ones not in exclude list
for col in ebc_n.columns:
    if col not in exclude_cols:
        ebc_n[col] = pd.to_numeric(ebc_n[col], errors='coerce')

## Evidence-based funding core investments data frame

# Read in a copy of the data frame to keep the original intact.

eci_n = eci.copy()

# Create variable that combine position by position type across cost factors

eci_n["EBF Core Teachers Core Investments"] = eci_n.iloc[:, [22, 23, 24]].sum(axis=1)
eci_n["EBF Specialist Teachers Core Investments"] = eci_n.iloc[:, [27]].sum(axis=1)
eci_n["EBF Instructional Facilitators Core Investments"] = eci_n.iloc[:, [30, 31, 32]].sum(axis=1)
eci_n["EBF Core Intervention Teacher (Tutors) Core Investments"] = eci_n.iloc[:, [35, 36, 37]].sum(axis=1)
eci_n["EBF Core Investment Teacher Positions Core Investments"] = eci_n.iloc[:, [40]].sum(axis=1)
eci_n["EBF Special Population Teacher Positions Core Investments"] = eci_n.iloc[:, [41]].sum(axis=1)
eci_n["EBF Instructional Assistant Positions Core Investments"] = eci_n.iloc[:, [42]].sum(axis=1)
eci_n["EBF School Counselors Core Investments"] = eci_n.iloc[:, [45, 46, 47]].sum(axis=1)
eci_n["EBF School Nurses Core Investments"] = eci_n.iloc[:, [50, 51, 52]].sum(axis=1)
eci_n["EBF Supervisory Aides Core Investments"] = eci_n.iloc[:, [55, 56, 57]].sum(axis=1)
eci_n["EBF Librarians Core Investments"] = eci_n.iloc[:, [60, 61, 62]].sum(axis=1)
eci_n["EBF Librarian Aides Core Investments"] =  eci_n.iloc[:, [65, 66, 67]].sum(axis=1)
eci_n["EBF Principals Core Investments"] = eci_n.iloc[:, [70, 71, 72]].sum(axis=1)
eci_n["EBF Assistant Principals Core Investments"] = eci_n.iloc[:, [75, 76, 77]].sum(axis=1)
eci_n["EBF School Site Staff Core Investments"] = eci_n.iloc[:, [80, 81, 82]].sum(axis=1)

# Keep necessary columns

columns_to_keep = [0, 1] + list(range(89, 104))

eci_n = eci_n.iloc[:, columns_to_keep]

# Rename columns

eci_n.columns = ["RCDTS",
                 "District Name (EBF)",
                 "EBF Core Teachers Core Investments",
                 "EBF Specialist Teachers Core Investments",
                 "EBF Instructional Facilitators Core Investments",
                 "EBF Core Intervention Teacher (Tutors) Core Investments",
                 "EBF Core Investment Teacher Positions Core Investments",
                 "EBF Special Population Teacher Positions Core Investments",
                 "EBF Instructional Assistant Positions Core Investments",
                 "EBF School Counselors Core Investments",
                 "EBF School Nurses Core Investments",
                 "EBF Supervisory Aides Core Investments",
                 "EBF Librarians Core Investments",
                 "EBF Librarian Aides Core Investments",
                 "EBF Principals Core Investments",
                 "EBF Assistant Principals Core Investments",
                 "EBF School Site Staff Core Investments"]

# Change to numeric: Loop through columns and convert to numeric ones not in exclude list

# List of columns to exclude
exclude_cols = ["RCDTS","District Name (EBF)"]

# Loop through columns and convert the ones not in exclude list
for col in eci_n.columns:
    if col not in exclude_cols:
        eci_n[col] = pd.to_numeric(eci_n[col], errors='coerce')

# Add a statewide RCDTS value "6500000008000" which correpsonds to the IRC data 
# for the missing RCDTS value. NOTE: EBF data does not have a statewide RCDTS ID.

eci_n["RCDTS"] = eci_n["RCDTS"].fillna("6500000008000")

## Illinois Report Card data frame

# Read in a copy of the data frame to keep the original intact.

irc_n = irc.copy()

# Keep necessary columns

irc_n = irc_n[["RCDTS",
             "Level", # This was "Type" in SY24
             "District",
             "# Student Enrollment",
             "% Student Enrollment - White",
             "% Student Enrollment - Black or African American",
             "% Student Enrollment - Hispanic or Latino",
             "% Student Enrollment - Asian",
             "% Student Enrollment - Native Hawaiian or Other Pacific Islander",
             "% Student Enrollment - American Indian or Alaska Native",
             "% Student Enrollment - Two or More Races",
             "% Student Enrollment - Middle Eastern or North African",
             "% Student Enrollment - IEP",
             "% Student Enrollment - EL",
             "% Student Enrollment - Low Income",
             "# Student Enrollment - White",
             "# Student Enrollment - Black or African American",
             "# Student Enrollment - Hispanic or Latino",
             "# Student Enrollment - Asian",
             "# Student Enrollment - Native Hawaiian or Other Pacific Islander",
             "# Student Enrollment - American Indian or Alaska Native",
             "# Student Enrollment - Two or More Races",
             "# Student Enrollment - Middle Eastern or North African",
             "# Student Enrollment - IEP",
             "# Student Enrollment - EL",
             "# Student Enrollment - Low Income",
             "Total Teacher FTE",
             "School Counselor FTE",
             "School Nurse FTE",
             "School Psychologist FTE",
             "School Social Worker FTE"]]

# Change "Level" to "Type" tobe consistent with prior year

irc_n = irc_n.rename(columns={"Level": "Type"})

# NOTE: SY25 IRC RCDTS data has "-" between digits. Remove these first (for irc and rcf)

irc_n["RCDTS"] = irc_n["RCDTS"].astype(str).str.replace("-", "")
irc["RCDTS"] = irc["RCDTS"].astype(str).str.replace("-", "")
rcf["RCDTS"] = rcf["RCDTS"].astype(str).str.replace("-", "")

# Create a IRC school count variable (for positions per school calculations)

# Create a data frame of school counts by RCDTS

irc_school = irc.copy()
irc_school = irc_school[irc_school["Level"] == "School"] # Change "Type" to "Level"
# Change back to Type to keep consistent with prior year
irc_school = irc_school.rename(columns={"Level": "Type"})

irc_school = irc_school[["RCDTS","Type"]]
irc_school["RCDTS"] = irc_school["RCDTS"].astype(str).str[:11]+"0000"
irc_school = irc_school["RCDTS"].value_counts().reset_index()

# Merge with irc_n dataframe

irc_n = irc_n.merge(irc_school, how="left", on="RCDTS")

# Rename columns

irc_n.columns = ["RCDTS",
                 "District Type",
                 "District Name (IRC)",
                 "Student Enrollment (#)",
                 "White (%)",
                 "Black (%)",
                 "Latine (%)",
                 "Asian (%)",
                 "Native Hawaiian or Other Pacific Islander (%)",
                 "American Indian or Alaska Native (%)",
                 "Two or more races (%)",
                 "Middle Eastern or North African (%)",
                 "IEP (%)",
                 "EL (%)",
                 "Low Income (%)",
                 "White (#)",
                 "Black (#)",
                 "Latine (#)",
                 "Asian (#)",
                 "Native Hawaiian or Other Pacific Islander (#)",
                 "American Indian or Alaska Native (#)",
                 "Two or more races (#)",
                 "Middle Eastern or North African (#)",
                 "IEP (#)",
                 "EL (#)",
                 "Low Income (#)",     
                 "IRC Teacher FTE",
                 "IRC School Counselor FTE",
                 "IRC School Nurse FTE",
                 "IRC School Psychologist FTE",
                 "IRC School Social Worker FTE",
                 "School Count"]

# Format changes to IRC data frame

# NOTE: Per IRC business rules SY25 change missing to 0 and "*" to missing (i.e redacted)
#       Change data to numeric

# Change missing to 0

irc_n = irc_n.fillna(0)

# Change * to missing (i.e. redacted) - replace * with np.nan

irc_n = irc_n.replace('*', np.nan)

# Change to numeric: Loop through columns and convert to numeric ones not in exclude list
        
# List of columns to exclude
exclude_cols = ["RCDTS","District Type","District Name (IRC)"]

# Loop through columns and convert the ones not in exclude list
for col in irc_n.columns:
    if col not in exclude_cols:
        irc_n[col] = pd.to_numeric(irc_n[col], errors='coerce')
        
# Keep only "District" and "Statewide" in "District Type" column
# NOTE: This removes school-level columns, which are unnecessary for PEER app.

irc_n = irc_n[(irc_n["District Type"] == "District") | (irc_n["District Type"] == "Statewide")]

# Drop district type column.

irc_n = irc_n.drop(columns=["District Type"])

## IRC finance data 

# Read in a copy of the data frame to keep the original intact.

rcf_n = rcf.copy()

# Keep necessary columns

rcf_n = rcf_n[["RCDTS",
               "Level",
               "% Local Property Taxes",
               "% Other Local Funding",
               "% Evidence-Based Funding",
               "% Other State Funding",
               "% Federal Funding"
               ]]

# Rename columns

rcf_n.columns = ["RCDTS",
                 "District Type",
                 "Local Property Taxes (%)",
                 "Other Local Funding (%)",
                 "Evidence-Based Funding (%)",
                 "Other State Funding (%)",
                 "Federal Funding (%)"]

# Keep only "District" and "Statewide" in "District Type" column

rcf_n = rcf_n[(rcf_n["District Type"] == "District") | (rcf_n["District Type"] == "Statewide")]

# Drop district type column

rcf_n = rcf_n.drop(columns=["District Type"])

## Employment Information System data frame

# Read in a copy of the data frame to keep the original intact.
         
eis_z = eis.copy()

eis_z = eis_z.iloc[:, [1, #RCDTS 
                        6]] # PositionCodeDescription

# Change EIS data from long to wide format.

# Create a new data frame of a pivot table where PositionCodeDescription are the column names, the keys are the RCDTS, and the values are the counts of each PositionCodeDescription.

eis_n = pd.pivot_table(eis_z,
                        index = "RCDTS",
                        columns = "PositionCodeDescription",
                        aggfunc = len,
                        fill_value = 0).reset_index()

# Add EIS infront of all column names except for RCDTS

eis_n.columns = ["RCDTS"] + ["EIS " + str(col) for col in eis_n.columns[1:]]

# Create a RCDTS value for statewide values (the sum of all columns except RCDTS in eis_n)

# Calculate the sum of columns 1:45

sum_values = eis_n.iloc[:, 1:46].sum()

# Create a new row with RCDTS = "65000000080" and the calculated sums
# NOTE: Add a zero to make it 14 digits, so that the code in the next block pads
#       work properly

new_row = pd.DataFrame([["650000000800000"] + sum_values.tolist()], columns=eis_n.columns)

# Append the new row to eis_n

eis_n = pd.concat([eis_n, new_row], ignore_index=True)

In [ ]:
# Step 7 - Join data frames together

# NOTE: In order to make RCDTS IDs commensurate create a new
#       column for the IRC data that contains the first 13 
#       characters of the RCDTS. This will allow us to join the data sets together.
#       For simplicity, create a new column to base the join on.
#
#       RCDTS is 13 for all EBF data, 15 for IRC data. 
#       RCDTS for EIS data is 14 for some (for RCDTS IDs that begin with 0
#       the 0 is cut and) and 15 for others (that do not begin with 0).
#       For this data first pad the RCDTS with 0s to make it 15 characters
#       and then keep the first 13 digits.

# NOTE: Create technical documentation for RCDTS formatting in future.

# Use .loc to explicitly modify the DataFrame
eis_n.loc[:, "RCDTS"] = eis_n["RCDTS"].astype(str).str.zfill(15)
eis_n.loc[:, "RCDTS"] = eis_n["RCDTS"].astype(str).str[:13]
irc_n.loc[:, "RCDTS"] = irc_n["RCDTS"].astype(str).str[:13]
rcf_n.loc[:, "RCDTS"] = rcf_n["RCDTS"].astype(str).str[:13]
eai_n.loc[:, "RCDTS"] = eai_n["RCDTS"]
ebc_n.loc[:, "RCDTS"] = ebc_n["RCDTS"]
eci_n.loc[:, "RCDTS"] = eci_n["RCDTS"]

# Join data frames together using left join on RCDTS

df_x = irc_n.copy()
df_x = pd.merge(df_x, rcf_n, on="RCDTS", how="left")
df_x = pd.merge(df_x, ebc_n, on="RCDTS", how="left")
df_x = pd.merge(df_x, eai_n, on="RCDTS", how="left")
df_x = pd.merge(df_x, eci_n, on="RCDTS", how="left")
df_x = pd.merge(df_x, eis_n, on="RCDTS", how="left")
#df = pd.merge(df, dee_n, on="RCDTS", how="left")

# Change "District Name (IRC)" row where value is 0 to "Statewide"
 
df_x = df_x.replace({"District Name (IRC)": {0: "State of Illinois"}})

# Remove missing values from District Name (IRC) column

df_x = df_x.dropna(subset=["District Name (IRC)"])

In [ ]:
# Step 8 - Adequacy calculations

df = df_x.copy()

# Create funding, funding gap, and adequacy level calculations

# Total current resources
# NOTE: For readability changing current to actual.
df["Actual Resources"] = df["EBF Base Calc Final Resources"]
# Per pupil
df["Actual Resources Per Student"] = df["Actual Resources"]/df["Total ASE"]

# Adequacy target
df["Adequacy Target"] = df["EBF Base Calc Final Adequacy Target"]
# Per pupil
df["Adequacy Target Per Student"] = df["EBF Base Calc Final Adequacy Target Per Student"]

# Funding gap
df["Adequacy Funding Gap"] = df["EBF Base Calc Adequacy Funding Gap"]
# Per pupil
df["Adequacy Funding Gap Per Student"] = df["Adequacy Funding Gap"]/df["Total ASE"]

# Adequacy level
df["Adequacy Level"] = df["Actual Resources"]/df["Adequacy Target"]

# Create actual positions that match EBF adequacy language

# Positions gaps: Acutal - Adequate; + = more than adequate; - = less than adequate

# NOTE: For teacher definitions see IRC business rules (https://www.isbe.net/Documents/Public-Business-Rules-2024-Report-Card-Metrics.pdf, pg. 231 )
#       and EIS data elements (https://www.isbe.net/Pages/EIS-Data-Elements.aspx).

# Recalculate adequacy gap to match calculations below where negative values indicates a gap

# NOTE: Since SSP is phased out, will not perform gaps per school calcualtions.

df["Adequacy Funding Gap"] = df["Actual Resources"] - df["Adequacy Target"]

# Drop rows if df["Adequacy Funding Gap"] is NA


df = df[df["Adequacy Funding Gap"].notna()]


positive_gap_sum = df.loc[(df["Adequacy Funding Gap"] < 0) & (df["RCDTS"] != "6500000008000"), "Adequacy Funding Gap"].sum()

df.loc[df["RCDTS"] == "6500000008000", "Adequacy Funding Gap"] = positive_gap_sum

df["Adequacy Funding Gap Per School"] = df["Adequacy Funding Gap"] / df["School Count"]

df = df[df["Adequacy Funding Gap Per School"].notna()]

positive_gap_sum_perschool = df.loc[(df["Adequacy Funding Gap Per School"] < 0) & (df["RCDTS"] != "6500000008000"), "Adequacy Funding Gap Per School"].sum()
df.loc[df["RCDTS"] == "6500000008000", "Adequacy Funding Gap Per School"] = positive_gap_sum_perschool

# Teachers

# IRC teacher defition is regular and special education teacher per EIS definition.

# EBF adequacy

df["Adequate Core and Specialist Teachers"] = (df["EBF Core Teachers Core Investments"] + 
                           df["EBF Core Teacher Additional Investments"] + 
                           df["EBF Specialist Teachers Core Investments"] +
                           df["EBF Low Income Extended Day Teacher Additional Investments"] + 
                           df["EBF Low Income Summer School Teacher Additional Investments"]
                           )

df["Adequate Special Education Teachers"] = df["EBF Special Ed Teacher Additional Investments"]

# Actuals

df["Actual Core and Specialist Teachers Count (IRC)"] = df["IRC Teacher FTE"] # IRC teacher = regular and special education teacher per EIS definition
df["Actual Core and Specialist Teachers Count (EIS)"] = df["EIS Teacher"] + df["EIS Career and Technical Educator (CTE)"]
df["Actual Special Education Teachers Count (EIS)"] = df["EIS Special Education Teacher"]

# Gaps

df["Core and Specialist Teachers Gap (IRC)"] = df["Actual Core and Specialist Teachers Count (IRC)"] - (df["Adequate Core and Specialist Teachers"] + 
                                                            df["Adequate Special Education Teachers"])

df["Core and Specialist Teachers Gap (EIS)"] = df["Actual Core and Specialist Teachers Count (EIS)"] - df["Adequate Core and Specialist Teachers"]

# Recalculate State of Illinois so that it is the sum of positive gaps

# Recalculate State of Illinois so that it is the sum of positive gaps for teacher columns
# Ensure the column is numeric and NaNs are handled
df["Core and Specialist Teachers Gap (IRC)"] = pd.to_numeric(df["Core and Specialist Teachers Gap (IRC)"], errors="coerce").fillna(0)
df["Core and Specialist Teachers Gap (EIS)"] = pd.to_numeric(df["Core and Specialist Teachers Gap (EIS)"], errors="coerce").fillna(0)

for col in ["Core and Specialist Teachers Gap (IRC)", "Core and Specialist Teachers Gap (EIS)"]:
    positive_gap_sum = df.loc[(df[col] < 0) & (df["RCDTS"] != "6500000008000"), col].sum()
    df.loc[df["RCDTS"] == "6500000008000", col] = positive_gap_sum

# Gaps per school

# NOTE: Calculated per school for EIS only since IRC teacher FTE includes special education teachers.

df["Core and Specialist Teachers Gap Per School"] = df["Core and Specialist Teachers Gap (EIS)"] / df["School Count"]

# Gaps for special education teachers

df["Special Education Teachers Gap (EIS)"] = df["Actual Special Education Teachers Count (EIS)"] - df["Adequate Special Education Teachers"]

for col in ["Special Education Teachers Gap (EIS)"]:
    positive_gap_sum = df.loc[(df[col] < 0) & (df["RCDTS"] != "6500000008000"), col].sum()
    df.loc[df["RCDTS"] == "6500000008000", col] = positive_gap_sum

# Gaps per school

df["Special Education Teachers Gap Per School"] = df["Special Education Teachers Gap (EIS)"] / df["School Count"]

# Counselors

# EBF adequacy

df["Adequate Counselors"] = df["EBF School Counselors Core Investments"]

# Actuals

df["Actual Counselors Count (IRC)"] = df["IRC School Counselor FTE"]

# Gaps

df["Counselors Gap (IRC)"] = df["IRC School Counselor FTE"] - df["EBF School Counselors Core Investments"]

for col in ["Counselors Gap (IRC)"]:
    positive_gap_sum = df.loc[(df[col] < 0) & (df["RCDTS"] != "6500000008000"), col].sum()
    df.loc[df["RCDTS"] == "6500000008000", col] = positive_gap_sum

# Gaps per school

df["Counselors Gap Per School"] = df["Counselors Gap (IRC)"] / df["School Count"]

# Nurses

# EBF adequacy

df["Adequate Nurses"] = df["EBF School Nurses Core Investments"]

# Actuals

df["Actual Nurses Count (IRC)"] = df["IRC School Nurse FTE"]

# Gaps

df["Nurses Gap (IRC)"] = df["IRC School Nurse FTE"] - df["EBF School Nurses Core Investments"]

for col in ["Nurses Gap (IRC)"]:
    positive_gap_sum = df.loc[(df[col] < 0) & (df["RCDTS"] != "6500000008000"), col].sum()
    df.loc[df["RCDTS"] == "6500000008000", col] = positive_gap_sum

# Gaps per school

df["Nurses Gap Per School"] = df["Nurses Gap (IRC)"] / df["School Count"]

# Psychologists

# Adequacy

df["Adequate Psychologists"] = df["EBF Special Ed School Psychologist Additional Investments"] 

# Actuals

df["Actual Psychologists Count (IRC)"] = df["IRC School Psychologist FTE"]

# Gaps

df["Psychologists Gap (IRC)"] = df["IRC School Psychologist FTE"] - df["EBF Special Ed School Psychologist Additional Investments"]

for col in ["Psychologists Gap (IRC)"]:
    positive_gap_sum = df.loc[(df[col] < 0) & (df["RCDTS"] != "6500000008000"), col].sum()
    df.loc[df["RCDTS"] == "6500000008000", col] = positive_gap_sum

# Gaps per school

df["Psychologists Gap Per School"] = df["Psychologists Gap (IRC)"] / df["School Count"]

# Principal

# EBF adequacy

df["Adequate Principals"] = df["EBF Principals Core Investments"]

# Actuals

df["Actual Principals Count (EIS)"] = df["EIS Principal"]

# Gaps

df["Principals Gap (EIS)"] = df["EIS Principal"] - df["EBF Principals Core Investments"]

for col in ["Principals Gap (EIS)"]:
    positive_gap_sum = df.loc[(df[col] < 0) & (df["RCDTS"] != "6500000008000"), col].sum()
    df.loc[df["RCDTS"] == "6500000008000", col] = positive_gap_sum

# Gaps per school

df["Principals Gap Per School"] = df["Principals Gap (EIS)"] / df["School Count"]

# Assistant Principal

# EBF adequacy

df["Adequate Assistant Principals"] = df["EBF Assistant Principals Core Investments"]

# Actuals

df["Actual Assistant Principals Count (EIS)"] = df["EIS Assistant Principal"]

# Gaps

df["Assistant Principals Gap (EIS)"] = df["EIS Assistant Principal"] - df["EBF Assistant Principals Core Investments"]

for col in ["Assistant Principals Gap (EIS)"]:
    positive_gap_sum = df.loc[(df[col] < 0) & (df["RCDTS"] != "6500000008000"), col].sum()
    df.loc[df["RCDTS"] == "6500000008000", col] = positive_gap_sum

# Gaps per school

df["Assistant Principals Gap Per School"] = df["Assistant Principals Gap (EIS)"] / df["School Count"]

# English Learner

# EBF adequacy

df["Adequate EL Teachers"] = (df["EBF English Learner Intervention Teacher Additional Investments"] + 
                              df["EBF English Learner Extended Day Teacher Additional Investments"] + 
                              df["EBF English Learner Summer School Teacher Additional Investments"])

# Actuals

df["Actual EL Teachers (EIS)"] = df["EIS English as a Second Language Teacher"]

# Gap

df["EL Teachers Gap (EIS)"] = df["Actual EL Teachers (EIS)"] - df["Adequate EL Teachers"]

for col in ["EL Teachers Gap (EIS)"]:
    positive_gap_sum = df.loc[(df[col] < 0) & (df["RCDTS"] != "6500000008000"), col].sum()
    df.loc[df["RCDTS"] == "6500000008000", col] = positive_gap_sum

# Gaps per school

df["EL Teachers Gap Per School"] = df["EL Teachers Gap (EIS)"] / df["School Count"]

# Export a column name dataframe

dfcols = pd.DataFrame(df.columns)

dfcols.to_excel(rf"{file_path}\processed\main_column_names.xlsx")

In [ ]:
# Step 9 - Final formatting changes

df_reduced = df.copy()

# Reduce the data frame to necessary columns

df_reduced = df_reduced[['RCDTS',
                 'District Name (IRC)',
                 'School Count',
                 'Total ASE',
                 'White (%)', 
                 'Black (%)',
                 'Latine (%)', 
                 'Asian (%)',
                 'Native Hawaiian or Other Pacific Islander (%)',
                 'American Indian or Alaska Native (%)',
                 'IEP (%)', 
                 'EL (%)',
                 'Low Income (%)',
                 'Local Property Taxes (%)', 
                 'Other Local Funding (%)',
                 'Evidence-Based Funding (%)', 
                 'Other State Funding (%)',
                 'Federal Funding (%)', 
                 'Actual Resources',
                 'Adequacy Target',
                 'Adequacy Target Per Student', 
                 'Adequacy Funding Gap',
                 'Adequacy Funding Gap Per Student', 
                 'Adequacy Level',
                 'Adequate Core and Specialist Teachers',
                 'Actual Core and Specialist Teachers Count (EIS)',
                 'Adequate Special Education Teachers',
                 'Actual Special Education Teachers Count (EIS)', 
                 'Adequate EL Teachers',
                 'Actual EL Teachers (EIS)', 
                 'Adequate Counselors',
                 'Actual Counselors Count (IRC)',
                 'Adequate Nurses', 
                 'Actual Nurses Count (IRC)',
                 'Adequate Psychologists',
                 'Actual Psychologists Count (IRC)',
                 'Adequate Principals', 
                 'Actual Principals Count (EIS)',
                 'Adequate Assistant Principals',
                 'Actual Assistant Principals Count (EIS)',
                 'Core and Specialist Teachers Gap (EIS)',
                 'Special Education Teachers Gap (EIS)', 
                 'Counselors Gap (IRC)',
                 'Nurses Gap (IRC)',
                 'Psychologists Gap (IRC)',
                 'Principals Gap (EIS)', 
                 'Assistant Principals Gap (EIS)',
                 'EL Teachers Gap (EIS)', 
                 'Adequacy Funding Gap Per School',
                 'Core and Specialist Teachers Gap Per School',
                 'Special Education Teachers Gap Per School',
                 'Counselors Gap Per School', 
                 'Nurses Gap Per School',
                 'Psychologists Gap Per School', 
                 'Principals Gap Per School',
                 'Assistant Principals Gap Per School', 
                 'EL Teachers Gap Per School'
                 ]]

# Change format of race to a decimal

percentage_cols = ['White (%)', 'Black (%)', 'Latine (%)', 'Asian (%)',
                   'Native Hawaiian or Other Pacific Islander (%)',
                   'American Indian or Alaska Native (%)', 'IEP (%)', 
                   'EL (%)', 'Low Income (%)']

for col in percentage_cols:
    df_reduced[col] = df_reduced[col].astype(float) / 100

# Change format of revenue source to decimal

df_reduced["Local Property Taxes (%)"] = df_reduced["Local Property Taxes (%)"]/100
df_reduced["Other Local Funding (%)"] = df_reduced["Other Local Funding (%)"]/100
df_reduced["Evidence-Based Funding (%)"] = df_reduced["Evidence-Based Funding (%)"]/100
df_reduced["Other State Funding (%)"] = df_reduced["Other State Funding (%)"]/100
df_reduced["Federal Funding (%)"]= df_reduced["Federal Funding (%)"]/100


In [ ]:
# Step 10 - Export final data frame

# Export to parquet file
# Parquet files are more efficient for storage and faster to read in.

# Set save path where you want to store data

save_path = r'C:\Users\cdpou\Documents\Projects\peer_streamlit_app_old\app_data_wide.parquet'

df_reduced.to_parquet(rf"{save_path}") 